In [22]:
from sympy import *
k_iso, theta, delta =  symbols("k_iso, theta, delta")
Ex, Ey =  symbols("Ex, Ey")

In [23]:
Q      = Matrix([[cos(theta), sin(theta)],[-sin(theta), cos(theta)]]) 
E_cart = Matrix([[Ex],[Ey]]) 

In [24]:
# Transform E to material coordinates
E_rot  = Q*E_cart
E_rot

Matrix([
[ Ex*cos(theta) + Ey*sin(theta)],
[-Ex*sin(theta) + Ey*cos(theta)]])

In [25]:
# Flux in material coordinates (transverse)
K     = Matrix([[k_iso, 0],[0, k_iso/delta]])
q_rot = -K *E_rot
q_rot

Matrix([
[       -k_iso*(Ex*cos(theta) + Ey*sin(theta))],
[-k_iso*(-Ex*sin(theta) + Ey*cos(theta))/delta]])

In [26]:
# Transform back to Cartesian
q_cart = Q.transpose() * q_rot
q_cart

Matrix([
[-k_iso*(Ex*cos(theta) + Ey*sin(theta))*cos(theta) + k_iso*(-Ex*sin(theta) + Ey*cos(theta))*sin(theta)/delta],
[-k_iso*(Ex*cos(theta) + Ey*sin(theta))*sin(theta) - k_iso*(-Ex*sin(theta) + Ey*cos(theta))*cos(theta)/delta]])

In [30]:
# Cartesian conductivity tensor
kxx = -q_cart[0].diff(Ex)
kxy = -q_cart[0].diff(Ey)
kyx = -q_cart[1].diff(Ex)
kyy = -q_cart[1].diff(Ey)
print(julia_code(kxx))
print(julia_code(kxy))
print(julia_code(kyx))
print(julia_code(kyy))

k_iso .* cos(theta) .^ 2 + k_iso .* sin(theta) .^ 2 ./ delta
k_iso .* sin(theta) .* cos(theta) - k_iso .* sin(theta) .* cos(theta) ./ delta
k_iso .* sin(theta) .* cos(theta) - k_iso .* sin(theta) .* cos(theta) ./ delta
k_iso .* sin(theta) .^ 2 + k_iso .* cos(theta) .^ 2 ./ delta
